In [1]:
import os 
print("Current working directory:", os.getcwd())
os.chdir("/users/lanza/cvision/Polyps")

Current working directory: /users/lanza/cvision/Polyps/Kvasir-SEG


In [2]:
import os
import sys
import json
import glob
import random
import shutil
from pathlib import Path

import yaml
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
from PIL import Image
from ultralytics import YOLO




# PART 1: DATASET CONVERTER


In [7]:
class KvasirToYOLO:
    """Convert Kvasir JSON bbox format to YOLO format with train/val/test support."""

    def __init__(self, image_dir, json_path, output_dir, seed=42):
        self.image_dir = Path(image_dir)
        self.json_path = Path(json_path)
        self.output_dir = Path(output_dir)
        self.seed = seed

        with open(self.json_path, 'r') as f:
            self.annotations = json.load(f)

        if not isinstance(self.annotations, dict):
            raise ValueError("Annotations JSON must be a dict keyed by image ID.")

        self._has_test_split = False

    @staticmethod
    def _voc_to_yolo_bbox(bbox, img_width, img_height):
        """Convert Pascal VOC bbox to YOLO format with validation."""
        xmin = float(bbox['xmin'])
        ymin = float(bbox['ymin'])
        xmax = float(bbox['xmax'])
        ymax = float(bbox['ymax'])

        # Clamp to image bounds
        xmin = max(0.0, min(xmin, img_width - 1))
        ymin = max(0.0, min(ymin, img_height - 1))
        xmax = max(0.0, min(xmax, img_width - 1))
        ymax = max(0.0, min(ymax, img_height - 1))

        # Skip invalid boxes
        if xmax <= xmin or ymax <= ymin:
            return None

        x_center = (xmin + xmax) / 2.0
        y_center = (ymin + ymax) / 2.0
        width = xmax - xmin
        height = ymax - ymin

        # Normalize
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height

        # Final clamps
        x_center = max(0.0, min(1.0, x_center))
        y_center = max(0.0, min(1.0, y_center))
        width = max(1e-6, min(1.0, width))
        height = max(1e-6, min(1.0, height))
        # Return YOLO line as [class_id, x, y, w, h]; class_id is hardcoded 0 for 'polyp'
        return [0, x_center, y_center, width, height]

    def prepare_dataset(self, train_split=0.8, val_split=None, test_split=None, create_empty_labels=True):
        """Prepare dataset with train/val/test splits."""
        
        # Validate splits
        for name, v in [('train_split', train_split), ('val_split', val_split), 
                       ('test_split', test_split)]:
            if v is not None and not (0.0 <= v <= 1.0):
                raise ValueError(f"{name} must be in [0, 1].")
            if train_split + (val_split or 0) + (test_split or 0) > 1.0 + 1e-8:
                raise ValueError("Sum of split fractions cannot exceed 1.0")


        if val_split is None and test_split is not None:
            raise ValueError("If you set test_split, you must also set val_split.")

        if val_split is None:
            val = 1.0 - train_split
            test = 0.0
        else:
            if test_split is None:
                remaining = 1.0 - (train_split + val_split)
                val = val_split
                test = max(0.0, remaining)
            else:
                val = val_split
                test = test_split

        self._has_test_split = test > 0.0 + 1e-9

        # Create directories
        dirs = {
            'train_images': self.output_dir / 'images' / 'train',
            'val_images':   self.output_dir / 'images' / 'val',
            'train_labels': self.output_dir / 'labels' / 'train',
            'val_labels':   self.output_dir / 'labels' / 'val',
        }
        if self._has_test_split:
            dirs.update({
                'test_images': self.output_dir / 'images' / 'test',
                'test_labels': self.output_dir / 'labels' / 'test',
            })
        for d in dirs.values():
            d.mkdir(parents=True, exist_ok=True)

        # Shuffle and split
        image_ids = list(self.annotations.keys())
        rnd = random.Random(self.seed)
        rnd.shuffle(image_ids)

        # Compute split counts
        n = len(image_ids)
        n_train = int(n * train_split)  # floor by int()
        n_val = int(n * val)            # floor by int()
        # Test gets the remainder so that train+val+test sums exactly to n
        n_test = n - n_train - n_val if self._has_test_split else 0

        train_ids = image_ids[:n_train]
        val_ids   = image_ids[n_train:n_train + n_val]
        test_ids  = image_ids[n_train + n_val:] if self._has_test_split else []

        print(f"Processing images: total={n} | train={len(train_ids)}, "
              f"val={len(val_ids)}" + (f", test={len(test_ids)}" if self._has_test_split else ""))

        # Process splits
        self._process_split(train_ids, dirs['train_images'], dirs['train_labels'], 
                          create_empty_labels)
        self._process_split(val_ids, dirs['val_images'], dirs['val_labels'], 
                          create_empty_labels)
        if self._has_test_split:
            self._process_split(test_ids, dirs['test_images'], dirs['test_labels'], 
                              create_empty_labels)

        # Create YAML
        self._create_yaml()

        print("\n✓ Dataset prepared successfully!")
        print(f"  Output: {self.output_dir.resolve()}")
        print(f"  Train:  {len(train_ids)} images")
        print(f"  Val:    {len(val_ids)} images")
        if self._has_test_split:
            print(f"  Test:   {len(test_ids)} images")

    def _process_split(self, image_ids, img_dir, label_dir, create_empty_labels=True):
        """Process images for a specific split."""
        for img_id in image_ids:
            img_path = self.image_dir / f"{img_id}.jpg"
            if not img_path.exists():
                img_path = self.image_dir / f"{img_id}.png"
            if not img_path.exists():
                print(f"Warning: Image not found for {img_id}")
                continue

            with Image.open(img_path) as img:
                img_width, img_height = img.size

            shutil.copy(img_path, img_dir / img_path.name)

            annotation = self.annotations.get(img_id, {})
            bboxes = annotation.get('bbox', [])

            yolo_lines = []
            for bbox in bboxes:
                yolo_bbox = self._voc_to_yolo_bbox(bbox, img_width, img_height)
                if yolo_bbox is not None:
                    line = ' '.join(f"{v:.6f}" if i > 0 else str(int(v))
                                   for i, v in enumerate(yolo_bbox))
                    yolo_lines.append(line)

            label_path = label_dir / f"{img_path.stem}.txt"
            if yolo_lines:
                with open(label_path, 'w') as f:
                    f.write('\n'.join(yolo_lines) + '\n')
            else:
                if create_empty_labels:
                    open(label_path, 'a').close()

    def _create_yaml(self):
        """Create data.yaml configuration file."""
        data = {
            'path': str(self.output_dir.resolve()),
            'train': 'images/train',
            'val': 'images/val',
            'nc': 1,
            'names': ['polyp'],
        }
        if self._has_test_split:
            data['test'] = 'images/test'

        yaml_path = self.output_dir / 'data.yaml'
        with open(yaml_path, 'w') as f:
            yaml.dump(data, f, default_flow_style=False)
        print(f"  Created: {yaml_path}")

# PART 2: AUTOMATIC LEARNING RATE FINDER

In [8]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO

def auto_find_best_lr(
    data_yaml_path,
    model_size='n',
    img_size=640,
    batch_size=8,
    epochs=20,
    test_lr0_values=None,
    project='runs/lr_finder',
    weight_score_map50=0.7,
    weight_score_recall=0.3
):
    """
    Automatically find the best learning rate by testing multiple values.
    Returns the lr0 that gives best performance (weighted mAP50 + Recall).
    """

    # Safer default for mutable arg
    if test_lr0_values is None:
        test_lr0_values = [1e-4, 1e-3, 5e-3, 1e-2, 5e-2]

    # Device detection
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = 'mps'
        print("Using Apple Silicon GPU (MPS)")
    elif torch.cuda.is_available():
        device = 0
        print("Using NVIDIA GPU (CUDA)")
    else:
        device = 'cpu'
        print("Using CPU")

    print("\n" + "=" * 70)
    print("AUTOMATIC LEARNING RATE FINDER")
    print("=" * 70)
    print(f"Testing {len(test_lr0_values)} learning rates...")
    print(f"Device: {device}")
    print("=" * 70 + "\n")

    # Show grid explicitly (helps debug)
    print("LR grid:", [f"{float(v):.6g}" for v in test_lr0_values])

    results_data = []

    for idx, lr0_value in enumerate(test_lr0_values, start=1):
        lr0_value = float(lr0_value)
        run_name = f'lr_test_{idx:02d}_{lr0_value:.6g}'
        print(f"[{idx}/{len(test_lr0_values)}] → Testing lr0 = {lr0_value:.6g} (run: {run_name})...", flush=True)

        # Fresh model per trial
        model = YOLO(f'yolov8{model_size}.pt')

        # Default failed values in case of early exception
        map50 = 0.0
        recall = 0.0
        score = 0.0
        run_dir = Path(project) / run_name

        try:
            # Train with this LR (force optimizer so lr0 is respected)
            model.train(
                data=data_yaml_path,
                epochs=epochs,
                imgsz=img_size,
                batch=batch_size,
                lr0=lr0_value,
                lrf=0.1,
                optimizer='AdamW',       # critical: avoid optimizer='auto'
                warmup_epochs=2,
                patience=999,
                name=run_name,
                project=project,
                exist_ok=True,
                verbose=False,
                mosaic=0.0,
                mixup=0.0,
                flipud=0.5,
                fliplr=0.5,
                seed=0,
                device=device,
            )

            # Load best weights
            best_weights = run_dir / 'weights' / 'best.pt'
            if not best_weights.exists():
                raise FileNotFoundError(f"best.pt not found at {best_weights}")

            best_model = YOLO(str(best_weights))

            # Validate with your desired thresholds
            # NOTE: arg names are 'conf' and 'iou' per Ultralytics API
            val_metrics = best_model.val(
                data=data_yaml_path,
                imgsz=img_size,
                batch=batch_size,
                device=device,
                verbose=False,
                save=False  
            )

            # Extract metrics safely
            map50 = float(getattr(val_metrics.box, 'map50', 0.0))

            r = getattr(val_metrics.box, 'r', None)
            if r is not None:
                try:
                    recall = float(np.mean(r)) if hasattr(r, '__len__') else float(r)
                except Exception:
                    recall = 0.0
            if recall == 0.0:
                rd = getattr(val_metrics, 'results_dict', {}) or {}
                recall = float(rd.get('metrics/recall(B)', 0.0))

            # Composite score
            score = weight_score_map50 * map50 + weight_score_recall * recall

            print(f"  mAP@0.50: {map50:.4f} | Recall: {recall:.4f} | Score: {score:.4f}\n")

        except Exception as e:
            print(f"  ✗ Failed: {e}\n")

        finally:
            # Free memory between trials
            try:
                del model
            except:
                pass
            try:
                del best_model
            except:
                pass
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Append exactly once per trial (either real numbers or zeros)
            results_data.append({
                'lr0': lr0_value,
                'map50': map50,
                'recall': recall,
                'score': score,
                'run_dir': str(run_dir)
            })

    # ---- Determine best ----
    valid_results = [r for r in results_data if r['score'] > 0.0]
    if not valid_results:
        raise RuntimeError("No valid results produced; check your data and settings.")

    best_result = max(valid_results, key=lambda x: x['score'])
    best_lr0 = best_result['lr0']

    # ---- Plot results ----
    lr0_values = [r['lr0'] for r in valid_results]
    scores = [r['score'] for r in valid_results]
    maps = [r['map50'] for r in valid_results]
    recalls = [r['recall'] for r in valid_results]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    ax1.semilogx(lr0_values, scores, 'o-', linewidth=2, markersize=10, color='#3498db')
    ax1.axvline(best_lr0, color='red', linestyle='--', linewidth=2, label=f'Best lr0: {best_lr0:g}')
    ax1.set_xlabel('Initial Learning Rate (lr0)', fontsize=12)
    ax1.set_ylabel('Composite Score', fontsize=12)
    ax1.set_title('Learning Rate Performance', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend()

    ax2.semilogx(lr0_values, maps, 'o-', linewidth=2, label='mAP@0.50', color='#2ecc71')
    ax2.semilogx(lr0_values, recalls, 's-', linewidth=2, label='Recall', color='#e74c3c')
    ax2.axvline(best_lr0, color='red', linestyle='--', linewidth=2, alpha=0.5)
    ax2.set_xlabel('Initial Learning Rate (lr0)', fontsize=12)
    ax2.set_ylabel('Metric Value', fontsize=12)
    ax2.set_title('mAP vs Recall', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.legend()

    plt.tight_layout()
    plt.savefig('lr_finder_results.png', dpi=150, bbox_inches='tight')
    print(f"\n📊 Plot saved: lr_finder_results.png")

    # ---- Summary ----
    print("\n" + "=" * 70)
    print("LR FINDER RESULTS")
    print("=" * 70)
    print(f"\n🎯 BEST INITIAL LEARNING RATE (lr0): {best_lr0:g}")
    print(f"   mAP@0.50: {best_result['map50']:.4f}")
    print(f"   Recall:   {best_result['recall']:.4f}")
    print(f"   Score:    {best_result['score']:.4f}")

    print("\n📋 All Results:")
    print(f"{'lr0':<12} {'mAP@0.50':<12} {'Recall':<12} {'Score':<12} {'Run'}")
    print("-" * 70)
    for r in sorted(valid_results, key=lambda x: x['score'], reverse=True):
        marker = "⭐" if r['lr0'] == best_lr0 else "  "
        print(f"{marker} {r['lr0']:<10.6f} {r['map50']:<12.4f} {r['recall']:<12.4f} {r['score']:<12.4f} {r['run_dir']}")
    print("=" * 70)

    return best_lr0

# PART 3: TRAINING, EVALUATION, AND INFERENCE



In [ ]:
def train_yolo(data_yaml_path, model_size='n', epochs=100, img_size=640, 
               batch_size=8, workers=4, lr0=0.01):
    """Train YOLOv8 model with medical-optimized settings."""
    
    # Device detection
    if torch.backends.mps.is_available():
        device = 'mps'
        print("Using Apple Silicon GPU (MPS)")
    elif torch.cuda.is_available():
        device = 0
        print("Using NVIDIA GPU (CUDA)")
    else:
        device = 'cpu'
        print("Using CPU")

    model = YOLO(f'yolov8{model_size}.pt')

    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        name='polyp_detection',
        patience=20,
        save=True,
        device=device,
        workers=workers,
        optimizer='AdamW',
        
        # Learning rate settings
        lr0=lr0,
        lrf=0.1,
        cos_lr = True,
        warmup_epochs=5,
        warmup_momentum=0.8,
        momentum=0.937,
        weight_decay=0.001,
        dropout=0.1,
        multi_scale=True,
        # Medical imaging augmentations
        mosaic=0.0,
        mixup=0.0,
        copy_paste=0.0,
        erasing=0.0,
        hsv_h=0.015,
        hsv_s=0.3,
        hsv_v=0.3,
        degrees=10.0,
        translate=0.1,
        scale=0.2,
        perspective=0.0,
        shear=0.0,
        flipud=0.5,  # Enabled for endoscopy
        fliplr=0.5,
        augment= True,
        auto_augment = 'randaugment',
        copy_paste = 0.1,
        mixup =  0.1,       
        erasing =  0.2,     
        blur = 0.01,
        noise = 0.02
    )
    return model


def evaluate_model(model_path, data_yaml_path, split='val'):
    """Evaluate trained model on validation or test split."""
    model = YOLO(model_path)
    
    if split not in ('val', 'test'):
        raise ValueError("split must be 'val' or 'test'")

    metrics = model.val(data=data_yaml_path, split=split, conf= 0.5, iou=0.5)

    print(f"\n{'='*70}")
    print(f"EVALUATION RESULTS ON {split.upper()} SET")
    print(f"{'='*70}")
    
    try:
        print(f"mAP@0.50     : {metrics.box.map50:.4f}")
        print(f"mAP@0.50:0.95: {metrics.box.map:.4f}")
        if hasattr(metrics.box, 'p') and len(metrics.box.p) > 0:
            print(f"Precision    : {metrics.box.p[0]:.4f}")
        if hasattr(metrics.box, 'r') and len(metrics.box.r) > 0:
            print(f"Recall       : {metrics.box.r[0]:.4f}")
        print("="*70)
    except Exception as e:
        print(f"Note: Metric structure changed: {e}")

    return metrics


def predict_and_visualize(model_path, image_path, conf_threshold=0.5):
    """Run inference and visualize results."""
    model = YOLO(model_path)
    results = model(image_path, conf=conf_threshold, iou=0.5)

    for result in results:
        img_with_boxes = result.plot()
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f'Polyp Detection (confidence > {conf_threshold})', fontsize=14)
        plt.tight_layout()
        plt.show()

        if hasattr(result, 'boxes') and len(result.boxes) > 0:
            print(f"\n✓ Detected {len(result.boxes)} polyp(s):")
            for i, box in enumerate(result.boxes):
                if hasattr(box, 'conf'):
                    conf = float(box.conf[0].item())
                    print(f"  Polyp {i+1}: confidence = {conf:.3f}")
        else:
            print("\n✓ No polyps detected")

def predict_on_all_images(model_path, image_dir, data_yaml_path=None, conf_threshold=0.5, save_dir='predictions'):
    """
    Run inference on all images in a directory, save results, and compute detailed statistics.
    
    Args:
        model_path: Path to trained model
        image_dir: Directory with test or val images
        data_yaml_path: Optional path to data.yaml for mAP evaluation
        conf_threshold: Confidence threshold
        save_dir: Directory to save prediction images
    """
    from pathlib import Path
    import cv2
    import numpy as np
    from ultralytics import YOLO

    model = YOLO(model_path)
    image_dir = Path(image_dir)
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    label_dir = image_dir.parent.parent / 'labels' / image_dir.name
    image_files = list(image_dir.glob("*.jpg")) + list(image_dir.glob("*.png"))

    if not image_files:
        print(f"No images found in {image_dir}")
        return

    print(f"\n{'='*70}")
    print(f"RUNNING INFERENCE ON ALL {len(image_files)} IMAGES")
    print(f"{'='*70}\n")

    total_gt_polyps = 0
    total_pred_polyps = 0
    images_with_polyps = 0

    TP = FP = FN = TN = 0

    for img_path in image_files:
        label_path = label_dir / f"{img_path.stem}.txt"
        if label_path.exists():
            with open(label_path, 'r') as f:
                gt_boxes = [line for line in f if line.strip()]
                num_gt = len(gt_boxes)
        else:
            num_gt = 0

        results = model(str(img_path), conf=conf_threshold, iou=0.5, verbose=False)

        for result in results:
            img_with_boxes = result.plot()
            output_path = save_dir / f"pred_{img_path.name}"
            cv2.imwrite(str(output_path), img_with_boxes)

            if hasattr(result, 'boxes') and len(result.boxes) > 0:
                num_pred = len(result.boxes)
                total_pred_polyps += num_pred
                images_with_polyps += 1
                print(f"✓ {img_path.name}: {num_pred} polyp(s)")
            else:
                num_pred = 0
                print(f"  {img_path.name}: No polyps detected")

            # Contingency matrix update
            if num_gt > 0 and num_pred > 0:
                TP += 1
            elif num_gt == 0 and num_pred > 0:
                FP += 1
            elif num_gt > 0 and num_pred == 0:
                FN += 1
            elif num_gt == 0 and num_pred == 0:
                TN += 1

            total_gt_polyps += num_gt

    # Compute metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # mAP evaluation
    map50 = map5095 = None
    if data_yaml_path:
        metrics = model.val(data=data_yaml_path, split=image_dir.name, conf=conf_threshold, iou=0.5)
        map50 = float(getattr(metrics.box, 'map50', 0.0))
        map5095 = float(getattr(metrics.box, 'map', 0.0))

    # Summary
    print(f"\n{'='*70}")
    print(f"INFERENCE SUMMARY")
    print(f"{'='*70}")
    print(f"Total images:           {len(image_files)}")
    print(f"Images with polyps:     {images_with_polyps}")
    print(f"Images without polyps:  {len(image_files) - images_with_polyps}")
    print(f"Total ground truth polyps: {total_gt_polyps}")
    print(f"Total predicted polyps:    {total_pred_polyps}")
    print(f"\nContingency Matrix (image-level):")
    print(f"  TP (GT & Pred):        {TP}")
    print(f"  FP (No GT, Pred):      {FP}")
    print(f"  FN (GT, No Pred):      {FN}")
    print(f"  TN (No GT, No Pred):   {TN}")
    print(f"\nPrecision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1_score:.4f}")
    if map50 is not None:
        print(f"mAP@0.50:     {map50:.4f}")
        print(f"mAP@0.50–0.95:{map5095:.4f}")
    print(f"\nPredictions saved to:   {save_dir.resolve()}")
    print(f"{'='*70}")

    

def export_model(model_path, format='onnx'):
    """Export model to different formats."""
    model = YOLO(model_path)
    model.export(format=format)
    print(f"✓ Model exported to {format} format")




In [10]:
if __name__ == "__main__": # Main execution block, no automatic run on import
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)

    # ========== CONFIGURATION ==========
    IMAGE_DIR = "Kvasir-SEG/images"
    JSON_PATH = "Kvasir-SEG/kavsir_bboxes.json"
    OUTPUT_DIR = "kvasir_yolo_dataset"
    MODEL_SIZE = 'm'  # 'n', 's', 'm', 'l', 'x'
    BATCH_SIZE = 8
    EPOCHS = 30
    DATA_YAML = f"{OUTPUT_DIR}/data.yaml"
    
    

In [11]:
if __name__ == "__main__":   
    
    # ========== STEP 1: Dataset Preparation ==========
    print("\n" + "="*70)
    print("STEP 1: DATASET PREPARATION")
    print("="*70)
    
    if not os.path.isdir(IMAGE_DIR):
        print(f"[ERROR] IMAGE_DIR not found: {IMAGE_DIR}")
        sys.exit(1)
    if not os.path.isfile(JSON_PATH):
        print(f"[ERROR] JSON_PATH not found: {JSON_PATH}")
        sys.exit(1)

    converter = KvasirToYOLO(IMAGE_DIR, JSON_PATH, OUTPUT_DIR, seed=SEED)
    converter.prepare_dataset(train_split=0.7, val_split=0.2, test_split=0.1)  # 70/20/10 split




STEP 1: DATASET PREPARATION
Processing images: total=1000 | train=700, val=200, test=100
  Created: kvasir_yolo_dataset/data.yaml

✓ Dataset prepared successfully!
  Output: /users/lanza/cvision/Polyps/kvasir_yolo_dataset
  Train:  700 images
  Val:    200 images
  Test:   100 images


In [ ]:

if __name__ == "__main__":
    # ========== STEP 2: Find Best Learning Rate ==========
    print("\n" + "="*70)
    print("STEP 2: FINDING BEST LEARNING RATE")
    print("="*70)
    
    best_lr0 = auto_find_best_lr(
        data_yaml_path=DATA_YAML,
        model_size=MODEL_SIZE,
        img_size=640,
        batch_size=BATCH_SIZE,
        test_lr0_values=[1e-5, 1e-4, 1e-3, 5e-3, 1e-2, 5e-2],
        epochs=15
    )
    print("Best lr0 found:", best_lr0)


STEP 1: DATASET PREPARATION
Processing images: total=1000 | train=700, val=200, test=100
  Created: kvasir_yolo_dataset/data.yaml

✓ Dataset prepared successfully!
  Output: /users/lanza/cvision/Polyps/kvasir_yolo_dataset
  Train:  700 images
  Val:    200 images
  Test:   100 images

STEP 2: FINDING BEST LEARNING RATE
Using CPU

AUTOMATIC LEARNING RATE FINDER
Testing 6 learning rates...
Device: cpu

LR grid: ['1e-05', '0.0001', '0.001', '0.005', '0.01', '0.05']
[1/6] → Testing lr0 = 1e-05 (run: lr_test_01_1e-05)...
Ultralytics 8.3.217 🚀 Python-3.12.3 torch-2.9.0+cu128 CPU (Intel Xeon Gold 6252 CPU @ 2.10GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=kvasir_yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, d

In [ ]:
if __name__ == "__main__":
 # ========== STEP 3: Train Model ==========
    best_lr0 = 1e-4 # default in case STEP 2 is skipped
    print("\n" + "="*70)
    print("STEP 3: TRAINING MODEL")
    print("="*70)
    print(f"Using lr0 = {best_lr0}\n")
    
    model = train_yolo(
        data_yaml_path=DATA_YAML,
        model_size=MODEL_SIZE,
        epochs=EPOCHS,
        img_size=640, #try to set 800
        batch_size=BATCH_SIZE,
        lr0=1e-4 
    )

    # Find best model
    BEST_MODEL_PATH = "runs/detect/polyp_detection/weights/best.pt"
    if not os.path.isfile(BEST_MODEL_PATH):
        candidates = sorted(glob.glob("runs/detect/*/weights/best.pt"))
        if candidates:
            BEST_MODEL_PATH = candidates[-1]
            print(f"\n[INFO] Using model: {BEST_MODEL_PATH}")
        else:
            print("\n[ERROR] Could not find best.pt")
            sys.exit(1)
            

    # ========== STEP 5: Test on Sample Image ==========
    print("\n" + "="*70)
    print("STEP 5: TESTING ON SAMPLE IMAGE")
    print("="*70)

    # Check for test images first, fallback to val if needed
    test_images = glob.glob(f"{OUTPUT_DIR}/images/test/*.*")
    val_images = glob.glob(f"{OUTPUT_DIR}/images/val/*.*")

    if test_images:
        TEST_IMAGE = random.choice(test_images)
        print(f"\n✓ Found {len(test_images)} test images")
        print(f"Visualizing: {TEST_IMAGE}")
        predict_and_visualize(BEST_MODEL_PATH, TEST_IMAGE, conf_threshold=0.5)
    elif val_images:
        TEST_IMAGE = random.choice(val_images)
        print(f"\n[WARN] No test images found. Using validation set instead.")
        print(f"✓ Found {len(val_images)} val images")
        print(f"Visualizing: {TEST_IMAGE}")
        predict_and_visualize(BEST_MODEL_PATH, TEST_IMAGE, conf_threshold=0.5)
    else:
        print("[ERROR] No images found in test or val sets for visualization")

    # ========== STEP 5b: Run on ALL Test Images ==========
    print("\n" + "="*70)
    print("STEP 5b: RUNNING INFERENCE ON ALL TEST IMAGES")
    print("="*70)

    if test_images:
        predict_on_all_images(BEST_MODEL_PATH, f"{OUTPUT_DIR}/images/test", data_yaml_path=DATA_YAML, conf_threshold=0.5, save_dir='test_predictions')
    elif val_images:
        print("[WARN] No test images found. Running on validation set instead...")
        predict_on_all_images(BEST_MODEL_PATH, f"{OUTPUT_DIR}/images/val", data_yaml_path=DATA_YAML, conf_threshold=0.5, save_dir='val_predictions')
    else:
        print("[ERROR] No images found in test or val sets for batch inference")




STEP 3: TRAINING MODEL
Using lr0 = 0.0001

Using CPU
New https://pypi.org/project/ultralytics/8.3.218 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.217 🚀 Python-3.12.3 torch-2.9.0+cu128 CPU (Intel Xeon Gold 6252 CPU @ 2.10GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=kvasir_yolo_dataset/data.yaml, degrees=10.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosai

In [14]:
BEST_MODEL_PATH = "runs/detect/polyp_detection2/weights/best.pt"
# ========== STEP 4: Evaluate Model ==========
print("\n" + "="*70)
print("STEP 4: MODEL EVALUATION")
print("="*70)

try:
    metrics = evaluate_model(BEST_MODEL_PATH, DATA_YAML, split='test')
except Exception as e:
    print(f"\n[WARN] Test split unavailable ({e}), using validation set")
    metrics = evaluate_model(BEST_MODEL_PATH, DATA_YAML, split='val')



STEP 4: MODEL EVALUATION
Ultralytics 8.3.217 🚀 Python-3.12.3 torch-2.9.0+cu128 CPU (Intel Xeon Gold 6252 CPU @ 2.10GHz)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 165.9±43.5 MB/s, size: 56.3 KB)
val: Scanning /users/lanza/cvision/Polyps/kvasir_yolo_dataset/labels/test... 100 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 100/100 815.4it/s 0.1s1s
val: New cache created: /users/lanza/cvision/Polyps/kvasir_yolo_dataset/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 0.4it/s 20.0s3.7ss
                   all        100        105      0.941      0.905      0.932      0.757
Speed: 1.4ms preprocess, 193.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /users/lanza/cvision/Polyps/runs/detect/val7

EVALUATION RESULTS ON TEST SET
mAP@0.50     : 0.9321
mAP@0.50:0.95: 0.7566
Precision    : 0.9406
Rec

In [ ]:
if __name__ == "__main__":
    # ========== STEP 6: Export Model ==========
    print("\n" + "="*70)
    print("STEP 6: EXPORTING MODEL")
    print("="*70)

    export_model(BEST_MODEL_PATH, format='onnx')
    export_model(BEST_MODEL_PATH, format='tensorrt')

    print("\n" + "=" * 70)
    print("ALL STEPS COMPLETED SUCCESSFULLY!")
    print("=" * 70)

In [ ]:
BEST_MODEL_PATH = "runs/detect/polyp_detection2/weights/best.pt"
print("\n" + "="*70)
print("STEP 5: TESTING ON SAMPLE IMAGE")
print("="*70)

# Check for test images first, fallback to val if needed
test_images = glob.glob(f"{OUTPUT_DIR}/images/test/*.*")
val_images = glob.glob(f"{OUTPUT_DIR}/images/val/*.*")

if test_images:
    TEST_IMAGE = random.choice(test_images)
    print(f"\n✓ Found {len(test_images)} test images")
    print(f"Visualizing: {TEST_IMAGE}")
    predict_and_visualize(BEST_MODEL_PATH, TEST_IMAGE, conf_threshold=0.5)
elif val_images:
    TEST_IMAGE = random.choice(val_images)
    print(f"\n[WARN] No test images found. Using validation set instead.")
    print(f"✓ Found {len(val_images)} val images")
    print(f"Visualizing: {TEST_IMAGE}")
    predict_and_visualize(BEST_MODEL_PATH, TEST_IMAGE, conf_threshold=0.5)
else:
    print("[ERROR] No images found in test or val sets for visualization")

# ========== STEP 5b: Run on ALL Test Images ==========
print("\n" + "="*70)
print("STEP 5b: RUNNING INFERENCE ON ALL TEST IMAGES")
print("="*70)

if test_images:
    predict_on_all_images(BEST_MODEL_PATH, f"{OUTPUT_DIR}/images/test", data_yaml_path=DATA_YAML, conf_threshold=0.5, save_dir='test_predictions')
elif val_images:
    print("[WARN] No test images found. Running on validation set instead...")
    predict_on_all_images(BEST_MODEL_PATH, f"{OUTPUT_DIR}/images/val", data_yaml_path=DATA_YAML, conf_threshold=0.5, save_dir='val_predictions')
else:
    print("[ERROR] No images found in test or val sets for batch inference")




STEP 5b: RUNNING INFERENCE ON ALL TEST IMAGES



RUNNING INFERENCE ON ALL 100 TEST IMAGES

✓ cju2hgsptlfam0835o3b59h1o.jpg: 1 polyp(s)
✓ cju13hp5rnbjx0835bf0jowgx.jpg: 1 polyp(s)
✓ cju7er4kc2opa0801anuxc0eb.jpg: 1 polyp(s)
✓ cju8czvnztbf40871b4m7t78w.jpg: 1 polyp(s)
✓ cju890guyoiti098753yg6cdu.jpg: 1 polyp(s)
✓ cju2z45kuzf6d0988nz2c819m.jpg: 1 polyp(s)
✓ cju2qz06823a40878ojcz9ccx.jpg: 1 polyp(s)
✓ cju5ew4h9cqaf0818rrczkmqh.jpg: 1 polyp(s)
✓ cju87mrypnb1e0818scv1mxxg.jpg: 1 polyp(s)
✓ cju1h5w4wxajx0835mc954kxy.jpg: 1 polyp(s)
✓ cju0s690hkp960855tjuaqvv0.jpg: 1 polyp(s)
✓ cju6vucxvvlda0755j7msqnya.jpg: 1 polyp(s)
✓ cju15ptjtppz40988odsm9azx.jpg: 1 polyp(s)
✓ cju7d4jk723eu0817bqz2n39m.jpg: 1 polyp(s)
✓ cju5i39mreass0817au8p22zy.jpg: 1 polyp(s)
✓ cju76l27oyrw907551ri2a7fl.jpg: 1 polyp(s)
✓ cju8b5p40r2c60987ofa0mu03.jpg: 1 polyp(s)
✓ cju5c5xc7algd0817pb1ej5yo.jpg: 1 polyp(s)
✓ cju34m7h536wq0988xz7gx79v.jpg: 1 polyp(s)
✓ cju7d7aut2a2p0818z4uxc6cd.jpg: 1 polyp(s)
✓ cju2rnkt22xep0801as160g9t.jpg: 2 polyp(s)
✓ cju8a56vxpy780850r45yu4wk.jpg: 